In [1]:
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from time import sleep
from datetime import date
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import pickle
import tqdm
import warnings
warnings.filterwarnings('ignore')
import requests
import re
from bs4 import BeautifulSoup
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window
import sys
sys.path.append("../")
import src.herramientas_scrapeo as hs
sys.path.append("../")
import src.biblioteca as bb

c:\Users\Lin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Solo la primera vez
productos_mercadona = bb.productos_mercadona_estructura.copy()
recorrer = bb.listacategorias_merc.copy()
errores = set()

In [3]:
#segunda vez sin kernel
dfincompleto = pd.read_csv("../data/merc2023-01-19_suma.csv")
for cat_id in dfincompleto["category_id"].unique():
    if cat_id in recorrer:
        recorrer.remove(cat_id)
    else:
        pass

In [4]:
hs.scrap_mercadona(recorrer, productos_mercadona, errores)

In [5]:
productos_mercadona

{'name': ['Pechuga de pavo bajo en sal Hacendado finas lonchas',
  'Pechuga de pavo bajo en sal Hacendado finas lonchas',
  'Pechuga de pavo Hacendado finas lonchas',
  'Pechuga de pavo finas lonchas Hacendado',
  'Salchichón de pavo Hacendado lonchas',
  'Pechuga de pavo 0% m.g Hacendado al corte',
  'Lomo de pavo Hacendado lonchas',
  'Longaniza de pavo extra Hacendado lonchas',
  'Tiras de pechuga pollo al horno Hacendado',
  'Jamón de pavo Hacendado finas lonchas',
  'Dados de pavo Hacendado',
  'Tiras de pechuga de pollo al natural Hacendado',
  'Longaniza de pollo extra Hacendado sticks para picar',
  'Pechuga de pollo braseada Hacendado finas lonchas',
  'Mortadela de pavo Hacendado finas lonchas',
  'Pechuga de pavo al horno de leña Campofrío lonchas',
  'Pechuga de pollo Hacendado finas lonchas',
  'Pechuga de pavo 0% m.g Hacendado',
  'Pechuga de pollo Hacendado al corte',
  'Pechuga de pavo natural Noel lonchas',
  'Pechuga de pavo Noel lonchas muy finas',
  'Pollo relleno a

In [6]:
dfm = pd.DataFrame(productos_mercadona)


In [7]:
dfm2 = dfm.copy()

In [8]:
for error in errores:
    dfm2 = dfm2[dfm2["category_id"] != error]

In [9]:
dfmcate_id = dfm2["category_id"].value_counts().reset_index()

In [10]:
def revisar_errores(col1, col2):
    if col2 < 5:
        if col1 in [235, 782]:
            pass
        else:
            errores.add(col1)
    else:
        pass


In [11]:
dfmcate_id.apply(lambda x: revisar_errores(x["index"], x["category_id"]), axis = 1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
dtype: object

In [16]:
len(errores)

49

In [12]:
for error in errores:
    dfm2 = dfm2[dfm2["category_id"] != error]

In [13]:
dfm2["category_id"].value_counts()

203    120
241     67
145     57
132     54
189     54
51      51
48      46
149     44
154     44
56      35
201     35
50      35
58      33
150     32
237     32
81      32
88      30
202     30
129     28
238     27
884     27
152     26
90      25
84      24
97      24
130     22
205     22
239     20
83      20
86      18
148     15
89      15
151     13
147     12
98      12
789     11
79       6
Name: category_id, dtype: int64

In [14]:
dfm2.dropna(inplace=True)

In [17]:
today = date.today()
dfsuma = pd.read_csv("../data/merc2023-01-20_suma.csv")
dfsuma2 = pd.concat([dfsuma, dfm2], axis = 0, join = "outer", ignore_index = True)
dfsuma2.to_csv(f"../data/merc{today}_suma.csv")